# Agno Tutorial: Building Your First AI Agent

Welcome to this tutorial on Agno! Agno is a lightweight and powerful Python framework for building AI agents. It's designed to be fast, flexible, and easy to use, allowing you to create everything from simple chatbots to complex multi-agent systems.

In this notebook, we'll cover the essential concepts of Agno, including:

* **Installation and Setup**: Getting your environment ready.
* **Creating a Basic Agent**: Your first "Hello, World!" with Agno.
* **Using Tools**: Giving your agent new abilities.
* **Building a Multi-Agent Team**: Making agents collaborate.
* **Asynchronous Operations**: Running agents concurrently.

## 1. Installation and Setup

First, let's install the necessary libraries. We'll use `pip` to install `agno` and `python-dotenv` for managing our API keys.

In [2]:
#!pip install -qU agno python-dotenv openai duckduckgo-search -q

### Loading API Keys from .env

It's a best practice to store your API keys in a `.env` file to keep them secure. Create a file named `.env` in the same directory as this notebook and add your OpenAI API key to it:

Now, let's load the API key from the `.env` file.

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## 2. Creating a Basic Agent

An **Agent** is the fundamental building block in Agno. It combines a large language model (LLM) with instructions, tools, and memory. Let's create a simple agent that acts as a cheerful AI assistant.

In [4]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# 1. Create the Agent
chatbot = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    description="You are a cheerful AI assistant!",
    markdown=True
)

# 2. Get a response
chatbot.print_response("Hello, Agno!", stream=True)

Output()

## 3. Using Tools

**Tools** extend the capabilities of your agents, allowing them to interact with external services and data. Agno has many pre-built tools, and you can also create your own. Here, we'll use the `DuckDuckGoTools` to give our agent web search abilities.

In [5]:
from agno.tools.duckduckgo import DuckDuckGoTools

# 1. Create an agent with a tool
researcher = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are a helpful research assistant.",
    tools=[DuckDuckGoTools()],
    markdown=True
)

# 2. Ask a question that requires a search
researcher.print_response("What is the latest news about AI agent?", stream=True)

Output()

/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/site-packages/agno/tools/duckduckgo.py:90: RuntimeWarning: This
package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(

## 4. Building a Multi-Agent Team

Agno excels at creating **multi-agent systems** where different agents can collaborate to solve a problem. Let's create a team with a `Capital` agent and an `Weather` agent.

In [19]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.team import Team
import requests
import warnings
import asyncio
import gc

# Silence noisy unclosed socket warnings from third‑party async clients
warnings.filterwarnings("ignore", category=ResourceWarning)

# 1) Capital Agent: use the model to determine the capital city
capital_agent = Agent(
    name="capital_name",
    model=OpenAIChat(id="gpt-4o-mini"),
    description=(
        "You know capital cities. When given a country name, respond with only the capital city name (no extra words)."
    ),
)

# 2) Weather tool: single function that geocodes a city then fetches its current weather

def weather_tool(city: str) -> dict:
    """
    Look up the city's coordinates (Open‑Meteo Geocoding API) and then fetch current weather
    (Open‑Meteo Forecast API). Returns a dict like:
    { city, latitude, longitude, temperature, windspeed, weathercode }.
    """
    # Geocode the city -> latitude/longitude
    geocode_url = "https://geocoding-api.open-meteo.com/v1/search"
    with requests.Session() as s:
        r = s.get(
            geocode_url,
            params={"name": city, "count": 1, "language": "en", "format": "json"},
            timeout=10,
        )
        r.raise_for_status()
        data = r.json() or {}
    results = data.get("results") or []
    if not results:
        return {}
    top = results[0]
    name = top.get("name") or city
    lat = float(top.get("latitude"))
    lon = float(top.get("longitude"))

    # Fetch current weather for those coordinates
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {"latitude": lat, "longitude": lon, "current_weather": True}
    with requests.Session() as s:
        r = s.get(weather_url, params=params, timeout=10)
        r.raise_for_status()
        wdata = r.json() or {}
    current = wdata.get("current_weather") or {}

    return {
        "city": name,
        "latitude": lat,
        "longitude": lon,
        **current,
    }

# 3) Weather Agent: uses the single weather_tool function
weather_agent = Agent(
    name="weather",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[weather_tool],
    description=(
        "You fetch weather data using the provided tools. When given a city, call weather_tool(city). "
        "If given a country, first get its capital (from the other agent) then call weather_tool(capital). "
        "Return a concise sentence with temperature (°C), wind (km/h), and weather code."
    ),
    markdown=True,
)

team = Team(name="Capital and Weather Team", mode="coordinate", members=[capital_agent, weather_agent])

result = await team.arun("What is the weather in the capital of Japan?")
print(result.content)

INFO Setting default model to OpenAI Chat

The capital city of Japan is Tokyo. 

As for the current weather in Tokyo, it is clear with a temperature of 28.2°C and a wind speed of 3.8 km/h.


## 5. Launch your agent in the Agno UI

Open Agno Playground, add the endpoint http://localhost:7777/v1 as a model/server source, then chat and inspect tool calls, history, and responses to test faster.

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.playground import Playground
from agno.storage.sqlite import SqliteStorage
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools
import os
import dotenv
import uvicorn
import threading
import socket
import time

# Load env vars
dotenv.load_dotenv()

# Global server handles for clean restarts in notebooks
try:
    NB_UVICORN_SERVER
except NameError:
    NB_UVICORN_SERVER = None
    NB_UVICORN_THREAD = None

agent_storage: str = "tmp/agents.db"
os.makedirs(os.path.dirname(agent_storage), exist_ok=True)

web_agent = Agent(
    name="Web Agent",
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools()],
    instructions=["Always include sources"],
    storage=SqliteStorage(table_name="web_agent", db_file=agent_storage),
    add_datetime_to_instructions=True,
    add_history_to_messages=True,
    num_history_responses=5,
    markdown=True,
)

finance_agent = Agent(
    name="Finance Agent",
    model=OpenAIChat(id="gpt-4o"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True)],
    instructions=["Always use tables to display data"],
    storage=SqliteStorage(table_name="finance_agent", db_file=agent_storage),
    add_datetime_to_instructions=True,
    add_history_to_messages=True,
    num_history_responses=5,
    markdown=True,
)

playground_app = Playground(agents=[web_agent, finance_agent])
app = playground_app.get_app()

# Helpers

def _port_in_use(port: int, host: str = "127.0.0.1") -> bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(0.2)
        return s.connect_ex((host, port)) == 0


def _find_free_port(preferred: int = 7777, host: str = "127.0.0.1", attempts: int = 20) -> int | None:
    if not _port_in_use(preferred, host):
        return preferred
    for offset in range(1, attempts + 1):
        cand = preferred + offset
        if not _port_in_use(cand, host):
            return cand
    return None


def stop_playground():
    global NB_UVICORN_SERVER, NB_UVICORN_THREAD
    if NB_UVICORN_SERVER is not None:
        print("Stopping Agno Playground...")
        NB_UVICORN_SERVER.should_exit = True
        if NB_UVICORN_THREAD is not None and NB_UVICORN_THREAD.is_alive():
            NB_UVICORN_THREAD.join(timeout=3)
        NB_UVICORN_SERVER = None
        NB_UVICORN_THREAD = None
        print("Stopped.")


def start_playground(preferred_port: int = 7777, host: str = "127.0.0.1"):
    global NB_UVICORN_SERVER, NB_UVICORN_THREAD

    # Stop any prior server launched from this notebook
    if NB_UVICORN_SERVER is not None:
        stop_playground()

    port = _find_free_port(preferred_port, host)
    if port is None:
        print("No free port found near", preferred_port)
        print("Tip (macOS): !lsof -ti tcp:7777 | xargs kill -15")
        return

    config = uvicorn.Config(app, host=host, port=port, log_level="info")
    NB_UVICORN_SERVER = uvicorn.Server(config)

    def _run_server():
        NB_UVICORN_SERVER.run()

    NB_UVICORN_THREAD = threading.Thread(target=_run_server, daemon=True)
    NB_UVICORN_THREAD.start()

    # Wait briefly for the socket to bind before printing URL
    for _ in range(20):
        if _port_in_use(port, host):
            break
        time.sleep(0.1)
    print(f"Agno Playground running at http://{host}:{port}")


# Start server compatible with Jupyter
start_playground(preferred_port=7777, host="127.0.0.1")

/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/site-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/site-packages/uvicorn/protocols/websockets/websockets_impl.py:22: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [27565]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7777 (Press CTRL+C to quit)
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7777 (Press CTRL+C to quit)


Agno Playground running at http://127.0.0.1:7777


INFO:     127.0.0.1:53181 - "GET /v1/playground/status HTTP/1.1" 200 OK
INFO:     127.0.0.1:53183 - "GET /v1/playground/agents HTTP/1.1" 200 OK
INFO:     127.0.0.1:53183 - "GET /v1/playground/agents HTTP/1.1" 200 OK
INFO:     127.0.0.1:53179 - "OPTIONS /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/sessions?user_id=devon_5563 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53179 - "OPTIONS /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/sessions?user_id=devon_5563 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53183 - "GET /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/sessions?user_id=devon_5563 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53183 - "GET /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/sessions?user_id=devon_5563 HTTP/1.1" 200 OK


/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/site-packages/agno/storage/sqlite.py:201: SADeprecationWarning: The Column.copy() method is deprecated and will be removed in a future release. (deprecated since: 1.4)
  *[c.copy() for c in self.table.columns],
/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/site-packages/agno/storage/sqlite.py:201: SADeprecationWarning: The Column.copy() method is deprecated and will be removed in a future release. (deprecated since: 1.4)
  *[c.copy() for c in self.table.columns],
/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/site-packages/agno/storage/sqlite.py:201: SADeprecationWarning: The Column.copy() method is deprecated and will be removed in a future release. (deprecated since: 1.4)
  *[c.copy() for c in self.table.columns],
/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/site-packages/agno/storage/sqlite.py:201: SADeprecationWarning: The Column.copy() method is deprecated and will be removed in a future release. (deprecated sin

INFO:     127.0.0.1:53178 - "OPTIONS /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53179 - "POST /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53179 - "POST /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53248 - "OPTIONS /v1/playground/teams HTTP/1.1" 200 OK
INFO:     127.0.0.1:53248 - "OPTIONS /v1/playground/teams HTTP/1.1" 200 OK
INFO:     127.0.0.1:53250 - "GET /v1/playground/teams HTTP/1.1" 200 OK
INFO:     127.0.0.1:53250 - "GET /v1/playground/teams HTTP/1.1" 200 OK
INFO:     127.0.0.1:53248 - "OPTIONS /v1/playground/workflows HTTP/1.1" 200 OK
INFO:     127.0.0.1:53248 - "OPTIONS /v1/playground/workflows HTTP/1.1" 200 OK
INFO:     127.0.0.1:53250 - "GET /v1/playground/workflows HTTP/1.1" 200 OK
INFO:     127.0.0.1:53250 - "GET /v1/playground/workflows HTTP/1.1" 200 OK
INFO:     127.0.0.1:53250 - "GET /v1/playground/team

/Users/devon/.pyenv/versions/3.10.14/lib/python3.10/inspect.py:1716: ResourceWarning: unclosed resource <TCPTransport closed=False reading=False 0x7fc88f5e3860>
  class_dict = dict_attr.__get__(entry)["__dict__"]


INFO:     127.0.0.1:54310 - "OPTIONS /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54312 - "POST /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54312 - "POST /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54320 - "POST /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54320 - "POST /v1/playground/agents/e7fa5baf-76f4-4700-b7b6-9862c00790a7/runs HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [27565]


![Agno](public/products/agno.png)

## Conclusion

Congratulations! You've learned the basics of Agno and how to create single agents, use tools, build multi-agent teams, and run asynchronous operations. This is just the beginning of what you can do with Agno. For more advanced topics, check out the official [Agno documentation](https://docs.agno.com/).